<a href="https://colab.research.google.com/github/vincent4u/CE807_Text_Analytics/blob/main/look_into_this.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#importing importantant libraries
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
import pandas as pd
import numpy as np

student_id = 2314555
np.random.seed(student_id)

train_df = pd.read_csv('/content/train.csv')
#train_df.head(20)

val_df=pd.read_csv('/content/valid.csv')
#val_df.head(20)

test_df=pd.read_csv('/content/test.csv')
#test_df.head(20)

#print(train_df.columns)
#print(test_df.columns)
#print(train_df['comment'])

In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import numpy as np
import os
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix



import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor

from scipy.stats import pearsonr
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

def preprocess_text(text):
    # Remove specific tokens
    text = text.replace("SDATA_5 :", "").replace("EDATA_5", "").replace("NEWLINE_TOKEN", "")


    # Remove special characters and punctuation marks using regular expressions
    text = re.sub(r'[^\w\s]', '', text)

    # Remove URLs, email addresses, and phone numbers
    text = re.sub(r"http\S+|www\S+|\S+@\S+|\d{10}", "", text)

    # Remove numbers
    text = re.sub(r"\d+", "", text)

    # Remove non-ASCII characters
    text = text.encode("ascii", "ignore").decode()

    # Convert the text to lowercase
    text = text.lower()

    # Remove extra white spaces
    text = re.sub(r"\s+", " ", text)

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the tokens back into a single string
    preprocessed_text = " ".join(tokens)

    return preprocessed_text

# Preprocess the text
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

#print(train_df['preprocessed_comment'].tolist())

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text (assuming you have a preprocess_text function)
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

X_train = train_df['preprocessed_comment'].tolist()
y_train = train_df['toxicity'].tolist()

X_val = val_df['preprocessed_comment'].tolist()
y_val = val_df['toxicity'].tolist()

# Create the bag-of-words representation
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_val_bow = vectorizer.transform(X_val)

# Define the parameter grid for GridSearchCV
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0]
}

# Create the Naive Bayes classifier
clf = MultinomialNB()

# Perform GridSearchCV
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train_bow, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on the validation data
y_pred = best_model.predict(X_val_bow)

accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm.auto import tqdm

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text
def preprocess_text(text):
    # Implement your text preprocessing function here
    text = text.replace("SDATA_5 :", "").replace("EDATA_5", "").replace("NEWLINE_TOKEN", "")
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters and punctuation
    text = re.sub(r"http\S+|www\S+|\S+@\S+|\d{10}", "", text)  # Remove URLs, email addresses, and phone numbers
    text = re.sub(r"\d+", "", text)  # Remove numbers
    text = text.encode("ascii", "ignore").decode()  # Remove non-ASCII characters
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra white spaces

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stop words and apply stemming
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

tqdm.pandas()  # Enable progress_apply for pandas
train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)

# Create the LDA pipeline
lda_pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('lda', LatentDirichletAllocation(n_components=10, random_state=42))  # Adjust n_components as needed
])

# Fit and transform the data
X_train_lda = lda_pipeline.fit_transform(train_df['preprocessed_comment'])
X_val_lda = lda_pipeline.transform(val_df['preprocessed_comment'])

# Naive Bayes Classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_lda, train_df['toxicity'])

# Predict on validation set
y_pred = nb_classifier.predict(X_val_lda)

# Evaluate the model
accuracy = accuracy_score(val_df['toxicity'], y_pred)
precision = precision_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
recall = recall_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
f1 = f1_score(val_df['toxicity'], y_pred, average='macro')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


  0%|          | 0/8699 [00:00<?, ?it/s]

  0%|          | 0/2920 [00:00<?, ?it/s]

Accuracy: 0.87
Precision: 0.43
Recall: 0.50
F1 Score: 0.46


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#LDA model alone

import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from tqdm.auto import tqdm
from joblib import Parallel, delayed

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text
def preprocess_text(text):
    # Remove special characters, URLs, and numbers using regex
    text = re.sub(r'[^\w\s]|http\S+|www\S+|\S+@\S+|\d+', '', text)

    # Convert to lowercase and remove extra whitespace
    text = text.lower().strip()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Join the tokens back into a string
    return ' '.join(tokens)

# Preprocess the text in a single pass
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

# Create the CountVectorizer and LDA pipeline
 #Create the CountVectorizer and LDA pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('lda', LatentDirichletAllocation(n_components=10, random_state=42)),
    ('classifier', MultinomialNB())  # Add a classifier here
])

# Fit the pipeline
pipeline.fit(train_df['preprocessed_comment'], train_df['toxicity'])

# Predict on validation set
y_pred = pipeline.predict(val_df['preprocessed_comment'])

# Evaluate the model
accuracy = accuracy_score(val_df['toxicity'], y_pred)
precision = precision_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
recall = recall_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
f1 = f1_score(val_df['toxicity'], y_pred, average='macro')

print(f"accuracy = {accuracy:.2f}")
print(f"precision = {precision:.2f}")
print(f"recall = {recall:.2f}")
print(f"f1 = {f1:.2f}")


accuracy = 0.87
precision = 0.43
recall = 0.50
f1 = 0.46


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Naive Bayes  only

import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from tqdm.auto import tqdm
from joblib import Parallel, delayed
from sklearn.neural_network import MLPClassifier

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text
def preprocess_text(text):
    # Remove special characters, URLs, and numbers using regex
    text = re.sub(r'[^\w\s]|http\S+|www\S+|\S+@\S+|\d+', '', text)

    # Convert to lowercase and remove extra whitespace
    text = text.lower().strip()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Join the tokens back into a string
    return ' '.join(tokens)

# Preprocess the text in a single pass
tqdm.pandas()  # Enable progress_apply for pandas
train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)

# Create the CountVectorizer and LDA pipeline with MLP classifier
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('lda', LatentDirichletAllocation(n_components=10, random_state=42)),
    ('classifier', MLPClassifier(hidden_layer_sizes=(100,), max_iter=100, random_state=42))  # MLP classifier
])

# Fit the pipeline
pipeline.fit(train_df['preprocessed_comment'], train_df['toxicity'])

# Predict on validation set
y_pred = pipeline.predict(val_df['preprocessed_comment'])

# Evaluate the model
accuracy = accuracy_score(val_df['toxicity'], y_pred)
precision = precision_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
recall = recall_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
f1 = f1_score(val_df['toxicity'], y_pred, average='macro')

print(f"accuracy = {accuracy:.2f}")
print(f"precision = {precision:.2f}")
print(f"recall = {recall:.2f}")
print(f"f1 = {f1:.2f}")


  0%|          | 0/8699 [00:00<?, ?it/s]

  0%|          | 0/2920 [00:00<?, ?it/s]

accuracy = 0.87
precision = 0.43
recall = 0.50
f1 = 0.46


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#LDA and Logistic regression

import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm.auto import tqdm
from xgboost import XGBClassifier

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text
def preprocess_text(text):
    # Remove special characters, URLs, and numbers using regex
    text = re.sub(r'[^\w\s]|http\S+|www\S+|\S+@\S+|\d+', '', text)

    # Convert to lowercase and remove extra whitespace
    text = text.lower().strip()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Join the tokens back into a string
    return ' '.join(tokens)

# Preprocess the text in a single pass
tqdm.pandas()  # Enable progress_apply for pandas
train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)

# Create the LDA pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('lda', LatentDirichletAllocation(n_components=10, random_state=42))  # Adjust n_components as needed
])

# Fit and transform the data
X_train = pipeline.fit_transform(train_df['preprocessed_comment'])
X_val = pipeline.transform(val_df['preprocessed_comment'])

# Your training and validation labels
y_train = train_df['toxicity']
y_val = val_df['toxicity']

# Train Logistic Regression classifier
#classifier = LogisticRegression(random_state=42)

classifier = XGBClassifier(random_state=42)
classifier.fit(X_train, y_train)

# Predict on validation set
y_pred = classifier.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='macro', zero_division='warn')
recall = recall_score(y_val, y_pred, average='macro', zero_division='warn')
f1 = f1_score(y_val, y_pred, average='macro')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


  0%|          | 0/8699 [00:00<?, ?it/s]

  0%|          | 0/2920 [00:00<?, ?it/s]

Accuracy: 0.86
Precision: 0.49
Recall: 0.50
F1 Score: 0.47


In [ ]:

#CNN
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tqdm.auto import tqdm

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text
def preprocess_text(text):
    # Remove special characters, URLs, and numbers using regex
    text = re.sub(r'[^\w\s]|http\S+|www\S+|\S+@\S+|\d+', '', text)

    # Convert to lowercase and remove extra whitespace
    text = text.lower().strip()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Join the tokens back into a string
    return ' '.join(tokens)

# Preprocess the text in a single pass
tqdm.pandas()  # Enable progress_apply for pandas
train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)

# Tokenize the text and convert to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['preprocessed_comment'])
X_train_seq = tokenizer.texts_to_sequences(train_df['preprocessed_comment'])
X_val_seq = tokenizer.texts_to_sequences(val_df['preprocessed_comment'])

# Pad sequences to ensure uniform length
max_len = max(max(len(seq) for seq in X_train_seq), max(len(seq) for seq in X_val_seq))
X_train_padded = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_val_padded = pad_sequences(X_val_seq, maxlen=max_len, padding='post')

# Define the CNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_len))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, train_df['toxicity'], epochs=10, batch_size=128, validation_data=(X_val_padded, val_df['toxicity']))

# Predict on validation set
y_pred = (model.predict(X_val_padded) > 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(val_df['toxicity'], y_pred)
precision = precision_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
recall = recall_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
f1 = f1_score(val_df['toxicity'], y_pred, average='macro')

print(f"accuracy = {accuracy:.2f}")
print(f"precision = {precision:.2f}")
print(f"recall = {recall:.2f}")
print(f"f1 = {f1:.2f}")


  0%|          | 0/8699 [00:00<?, ?it/s]

  0%|          | 0/2920 [00:00<?, ?it/s]

Epoch 1/10
68/68 [==============================] - 22s 263ms/step - loss: 0.3965 - accuracy: 0.8658 - val_loss: 0.3894 - val_accuracy: 0.8688
Epoch 2/10
68/68 [==============================] - 11s 165ms/step - loss: 0.3718 - accuracy: 0.8700 - val_loss: 0.4162 - val_accuracy: 0.8688
Epoch 3/10
68/68 [==============================] - 11s 154ms/step - loss: 0.2750 - accuracy: 0.8831 - val_loss: 0.5285 - val_accuracy: 0.8651
Epoch 4/10
68/68 [==============================] - 10s 143ms/step - loss: 0.1394 - accuracy: 0.9444 - val_loss: 0.7582 - val_accuracy: 0.8212
Epoch 5/10
68/68 [==============================] - 12s 178ms/step - loss: 0.0664 - accuracy: 0.9779 - val_loss: 0.9438 - val_accuracy: 0.7764
Epoch 6/10
68/68 [==============================] - 8s 116ms/step - loss: 0.0357 - accuracy: 0.9893 - val_loss: 1.1387 - val_accuracy: 0.8291
Epoch 7/10
68/68 [==============================] - 7s 110ms/step - loss: 0.0207 - accuracy: 0.9947 - val_loss: 1.1996 - val_accuracy: 0.7699
E

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from tqdm.auto import tqdm
from xgboost import XGBClassifier

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text
def preprocess_text(text):
    # Remove special characters, URLs, and numbers using regex
    text = re.sub(r'[^\w\s]|http\S+|www\S+|\S+@\S+|\d+', '', text)

    # Convert to lowercase and remove extra whitespace
    text = text.lower().strip()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Join the tokens back into a string
    return ' '.join(tokens)

# Preprocess the text in a single pass
tqdm.pandas()  # Enable progress_apply for pandas
train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)

# Create the pipeline with TF-IDF vectorizer and XGBoost classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', XGBClassifier())
])

# Fit the pipeline
pipeline.fit(train_df['preprocessed_comment'], train_df['toxicity'])

# Predict on validation set
y_pred = pipeline.predict(val_df['preprocessed_comment'])

# Evaluate the model
accuracy = accuracy_score(val_df['toxicity'], y_pred)
precision = precision_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
recall = recall_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
f1 = f1_score(val_df['toxicity'], y_pred, average='macro')

print(f"accuracy = {accuracy:.2f}")
print(f"precision = {precision:.2f}")
print(f"recall = {recall:.2f}")
print(f"f1 = {f1:.2f}")


  0%|          | 0/8699 [00:00<?, ?it/s]

  0%|          | 0/2920 [00:00<?, ?it/s]

accuracy = 0.87
precision = 0.53
recall = 0.50
f1 = 0.47


In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm.auto import tqdm
from xgboost import XGBClassifier

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text
def preprocess_text(text):
    # Remove special characters, URLs, and numbers using regex
    text = re.sub(r'[^\w\s]|http\S+|www\S+|\S+@\S+|\d+', '', text)

    # Convert to lowercase and remove extra whitespace
    text = text.lower().strip()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Join the tokens back into a string
    return ' '.join(tokens)

# Preprocess the text in a single pass
tqdm.pandas()  # Enable progress_apply for pandas
train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)

# Define the pipeline with TF-IDF vectorizer and XGBoost classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', XGBClassifier())
])

# Define parameter grid for GridSearchCV
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # ngram range
    'classifier__n_estimators': [100, 200, 300],  # number of trees
    'classifier__max_depth': [3, 5, 7],  # maximum depth of a tree
    'classifier__learning_rate': [0.01, 0.1, 0.3]  # learning rate
}

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy')
grid_search.fit(train_df['preprocessed_comment'], train_df['toxicity'])

# Predict on validation set
y_pred = grid_search.predict(val_df['preprocessed_comment'])

# Evaluate the model
accuracy = accuracy_score(val_df['toxicity'], y_pred)
precision = precision_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
recall = recall_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
f1 = f1_score(val_df['toxicity'], y_pred, average='macro')

print(f"accuracy = {accuracy:.2f}")
print(f"precision = {precision:.2f}")
print(f"recall = {recall:.2f}")
print(f"f1 = {f1:.2f}")


  0%|          | 0/8699 [00:00<?, ?it/s]

  0%|          | 0/2920 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from tqdm.auto import tqdm
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text
def preprocess_text(text):
    # Remove special characters, URLs, and numbers using regex
    text = re.sub(r'[^\w\s]|http\S+|www\S+|\S+@\S+|\d+', '', text)

    # Convert to lowercase and remove extra whitespace
    text = text.lower().strip()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Join the tokens back into a string
    return ' '.join(tokens)

# Preprocess the text in a single pass
tqdm.pandas()  # Enable progress_apply for pandas
train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)

# Create the pipeline with TF-IDF vectorizer and XGBoost classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', XGBClassifier())
])

# Define parameter grid for hyperparameter tuning
best_params = {
    "objective": "reg:squarederror",
    "n_estimators": 649,
    "verbosity": 0,
    "learning_rate": 0.029624377829993845,
    "max_depth": 5,
    "subsample":  0.9815939695298614,
    "colsample_bytree": 0.5082728972365318,
    "min_child_weight": 8
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(train_df['preprocessed_comment'], train_df['toxicity'])

# Get best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Predict on validation set
y_pred = best_model.predict(val_df['preprocessed_comment'])

# Evaluate the model
accuracy = accuracy_score(val_df['toxicity'], y_pred)
precision = precision_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
recall = recall_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
f1 = f1_score(val_df['toxicity'], y_pred, average='macro')

print("Best Hyperparameters:", best_params)
print(f"accuracy = {accuracy:.2f}")
print(f"precision = {precision:.2f}")
print(f"recall = {recall:.2f}")
print(f"f1 = {f1:.2f}")


In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from tqdm.auto import tqdm
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import StackingClassifier

# Load the data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Preprocess the text
def preprocess_text(text):
    # Remove special characters, URLs, and numbers using regex
    text = re.sub(r'[^\w\s]|http\S+|www\S+|\S+@\S+|\d+', '', text)

    # Convert to lowercase and remove extra whitespace
    text = text.lower().strip()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Join the tokens back into a string
    return ' '.join(tokens)

# Preprocess the text in a single pass
tqdm.pandas()  # Enable progress_apply for pandas
train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)

# Create the individual classifiers
xgb_classifier = XGBClassifier()
nb_classifier = MultinomialNB()

# Create the stacking classifier
stacking_classifier = StackingClassifier(
    estimators=[('xgb', xgb_classifier), ('nb', nb_classifier)],
    final_estimator=XGBClassifier()  # You can adjust the final estimator as needed
)

# Create the pipeline with TF-IDF vectorizer and stacking classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', stacking_classifier)
])

# Fit the pipeline
pipeline.fit(train_df['preprocessed_comment'], train_df['toxicity'])

# Predict on validation set
y_pred = pipeline.predict(val_df['preprocessed_comment'])

# Evaluate the model
accuracy = accuracy_score(val_df['toxicity'], y_pred)
precision = precision_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
recall = recall_score(val_df['toxicity'], y_pred, average='macro', zero_division='warn')
f1 = f1_score(val_df['toxicity'], y_pred, average='macro')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


  0%|          | 0/8699 [00:00<?, ?it/s]

  0%|          | 0/2920 [00:00<?, ?it/s]

Accuracy: 0.86
Precision: 0.45
Recall: 0.50
F1 Score: 0.46


In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 9.0 MB/s eta 0:00:00
